In [ ]:
from helpers.db.db_setup_methods import *
from helpers.db.db_query_methods import *

from helpers.db.db_helper_methods import *

init_db(2000000)


This is just to play with the group by.

First i would expect if i made like a group by that maybe if i had a index on it, it would be faster

In [ ]:
drop_non_clustered_indexes("products", get_database_name())

query = """
SELECT COUNT(*) 
FROM products
WHERE price = 999 
GROUP BY price;
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN " + query, database=get_database_name(), print_as_df=True, show_metrics=False)


In [ ]:
drop_non_clustered_indexes("products", get_database_name())

query = """
CREATE INDEX idx_price ON products (price);
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=False)

query = """
SELECT COUNT(*) 
FROM products 
WHERE price = 999 
GROUP BY price;
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN " + query, database=get_database_name(), print_as_df=True, show_metrics=False)

The index making it faster did seem to be the case.

Now i want to explore abit about if you also add a access predicate that is different from the group by what happens. 

I expect that no matter the order that the index would make it faster. But speed would also depend on the order where in this case i would think that since like the price = 999 will limit the search result the most then price, size would be the faster index

In [ ]:
drop_non_clustered_indexes("products", get_database_name())

query = """
SELECT size, COUNT(*) as count
FROM products
WHERE price = 999 
GROUP BY size;
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN " + query, database=get_database_name(), print_as_df=True, show_metrics=False)


In [ ]:
drop_non_clustered_indexes("products", get_database_name())

query = """
CREATE INDEX idx_price_size ON products (price, size);
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=False)

query = """
SELECT size, COUNT(*) as count
FROM products
WHERE price = 999 
GROUP BY size;
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN " + query, database=get_database_name(), print_as_df=True, show_metrics=False)


In [ ]:
drop_non_clustered_indexes("products", get_database_name())

query = """
CREATE INDEX idx_size_price ON products (size, price);
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=False)

query = """
SELECT size, COUNT(*) as count
FROM products
WHERE price = 999 
GROUP BY size;
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN " + query, database=get_database_name(), print_as_df=True, show_metrics=False)

I think i was right in my assumption. Though the difference in speed if i had the index size, price was not notciable at lower data amount but at 2M it was clear.